In [1]:
import pandas as pd
from pathlib import Path

In [2]:
project_root = Path().resolve().parent
onspd_file = project_root / "data" / "onspd-ppd" / "ONSPD_FEB_2025_UK.csv" # onspd csv avaialable on onedrive
pp_dir = project_root / "data" / "onspd-ppd" # download the ppd csvs between 2021-2025

In [3]:
tmp = pd.read_csv(onspd_file, nrows=0)
print(tmp.columns.tolist())

['pcd', 'pcd2', 'pcds', 'dointr', 'doterm', 'oscty', 'ced', 'oslaua', 'osward', 'parish', 'usertype', 'oseast1m', 'osnrth1m', 'osgrdind', 'oshlthau', 'nhser', 'ctry', 'rgn', 'streg', 'pcon', 'eer', 'teclec', 'ttwa', 'pct', 'itl', 'statsward', 'oa01', 'casward', 'npark', 'lsoa01', 'msoa01', 'ur01ind', 'oac01', 'oa11', 'lsoa11', 'msoa11', 'wz11', 'sicbl', 'bua24', 'ru11ind', 'oac11', 'lat', 'long', 'lep1', 'lep2', 'pfa', 'imd', 'calncv', 'icb', 'oa21', 'lsoa21', 'msoa21']


In [4]:
tmp_head = pd.read_csv(onspd_file, usecols=["pcds"], nrows=10) # input the name of the column you want to look into
print(tmp_head)

      pcds
0  AB1 0AA
1  AB1 0AB
2  AB1 0AD
3  AB1 0AE
4  AB1 0AF
5  AB1 0AG
6  AB1 0AJ
7  AB1 0AL
8  AB1 0AN
9  AB1 0AP


In [5]:
onspd_cols = [
    "pcds", # postcode w/o space
    "lsoa21",  # 2021 LSOA code
    "pfa", # police force area code
    "ctry", # country code (E92000001 = England)
    "rgn", # region code (E12000007 = London)
    "lat", # latitude
    "long", # longitude
    "imd" # deprivation score
]

In [6]:
onspd = (
    pd.read_csv(
        onspd_file, 
        usecols = onspd_cols, 
        dtype = str
    )
    .rename(columns = {
        "pcds":  "postcode", 
        "lsoa21": "LSOA_code", 
        "pfa": "police_force_area", 
        "ctry": "country_code", 
        "rgn": "region_code", 
        "lat": "latitude", 
        "long": "longitude", 
        "imd": "imd_score"
    })
    .assign(
        postcode = lambda df: (
            df.postcode.str.upper().str.replace(r"\s+", "", regex = True)
        ), 
        latitude = lambda df: pd.to_numeric(df.latitude, errors = "coerce"), 
        longitude = lambda df: pd.to_numeric(df.longitude, errors = "coerce"), 
        imd_score = lambda df: pd.to_numeric(df.imd_score, errors = "coerce")
    )
    # .query("region_code == 'E12000007'") # London
    .drop_duplicates(subset = "postcode") # ONSPD is supposed to have one unique postcode per row
    .reset_index(drop = True)
)

In [7]:
print(onspd.columns.tolist())

['postcode', 'country_code', 'region_code', 'latitude', 'longitude', 'police_force_area', 'imd_score', 'LSOA_code']


In [8]:
for path in sorted(pp_dir.glob("pp-*.csv")):
    hdrs = pd.read_csv(path, nrows=0).columns.tolist()
    print(path.name, hdrs)

pp-2021.csv ['{D707E535-5720-0AD9-E053-6B04A8C067CC}', '260000', '2021-08-06 00:00', 'SO45 2HT', 'T', 'N', 'F', '17', 'Unnamed: 8', 'PERRYWOOD CLOSE', 'HOLBURY', 'SOUTHAMPTON', 'NEW FOREST', 'HAMPSHIRE', 'A', 'A.1']
pp-2022.csv ['{045A1898-4ABF-9A24-E063-4804A8C048EA}', '407400', '2022-04-28 00:00', 'LU7 3FZ', 'S', 'Y', 'F', '68', 'Unnamed: 8', 'RAMSAY DRIVE', 'Unnamed: 10', 'LEIGHTON BUZZARD', 'CENTRAL BEDFORDSHIRE', 'CENTRAL BEDFORDSHIRE.1', 'A', 'A.1']
pp-2023.csv ['{0E082197-8499-5C09-E063-4704A8C0A10E}', '440000', '2023-10-12 00:00', 'B16 9BL', 'S', 'N', 'F', '46', 'Unnamed: 8', 'STIRLING ROAD', 'Unnamed: 10', 'BIRMINGHAM', 'BIRMINGHAM.1', 'WEST MIDLANDS', 'B', 'A']
pp-2024.csv ['{2F7F2B43-E776-E08F-E063-4804A8C05A49}', '185000', '2024-09-03 00:00', 'E6 1LP', 'F', 'N', 'L', '63A', 'Unnamed: 8', 'STAMFORD ROAD', 'Unnamed: 10', 'LONDON', 'NEWHAM', 'GREATER LONDON', 'B', 'A']
pp-2025.csv ['{31C68072-988E-FEE3-E063-4804A8C04F37}', '320000', '2025-02-28 00:00', 'KT20 5SJ', 'F', 'N', 'L

In [9]:
pp_files = sorted(pp_dir.glob("pp-202[1-5].csv")) 

In [10]:
pp_cols = [
    "txn_id",                   # Transaction unique identifier
    "price",                    # Price
    "date_of_transfer",         # Date of Transfer
    "postcode",                 # Postcode
    "property_type",            # Property Type
    "old_new",                  # Old / New
    "duration",                 # Duration
    "paon",                     # PAON
    "saon",                     # SAON
    "street",                   # Street
    "locality",                 # Locality
    "town_city",                # Town/City
    "district",                 # District
    "county",                   # County
    "ppd_category_type",        # PPD Category Type
    "record_status",            # Record Status
]

In [11]:
wanted = [
    "txn_id", 
    "postcode", 
    "price",
    "property_type",  
    "old_new", 
    "duration", 
    "ppd_category_type", 
    "record_status", 
]

In [12]:
def load_and_merge_pp(path):
    print(f"Loading: {path.name}")
    df = pd.read_csv(path, header = None, names = pp_cols, dtype=str)
    df["postcode"] = (
        df["postcode"]
        .str.upper()
        .str.replace(r"\s+", "", regex = True)
    )
    df = df[wanted]
    merged = df.merge(onspd, how = "left", on = "postcode")
    print(f"{len(merged):,} rows, and " f"{merged['LSOA_code'].isna().sum():,} missing LSOAs")
    return merged

In [13]:
print([f.name for f in pp_files])

['pp-2021.csv', 'pp-2022.csv', 'pp-2023.csv', 'pp-2024.csv', 'pp-2025.csv']


In [14]:
all_pp = pd.concat(
    (load_and_merge_pp(fp) for fp in pp_files),
    ignore_index = True
)

Loading: pp-2021.csv
1,277,243 rows, and 4,059 missing LSOAs
Loading: pp-2022.csv
1,068,645 rows, and 3,055 missing LSOAs
Loading: pp-2023.csv
845,990 rows, and 2,284 missing LSOAs
Loading: pp-2024.csv
737,255 rows, and 1,596 missing LSOAs
Loading: pp-2025.csv
106,006 rows, and 97 missing LSOAs


In [15]:
print(all_pp.columns.tolist())

['txn_id', 'postcode', 'price', 'property_type', 'old_new', 'duration', 'ppd_category_type', 'record_status', 'country_code', 'region_code', 'latitude', 'longitude', 'police_force_area', 'imd_score', 'LSOA_code']


In [16]:
all_pp[["txn_id", "LSOA_code", "police_force_area", "imd_score"]].isna().sum()

txn_id                   0
LSOA_code            11091
police_force_area    11091
imd_score            11088
dtype: int64

In [17]:
all_pp["LSOA_code"].isna().sum()

11091

In [18]:
all_pp = all_pp.dropna(subset=["LSOA_code"])

In [19]:
# 1a) How many rows & what percent
n = len(all_pp)
missing = all_pp["LSOA_code"].isna().sum()
print(f"{missing:,} / {n:,} rows missing → {missing/n:.2%}")

# 1b) Which postcodes didn’t match?
bad = all_pp.loc[all_pp["LSOA_code"].isna(), "postcode"]
print("Unique unmatched postcodes:", bad.nunique())
print(bad.unique()[:20])

0 / 4,024,048 rows missing → 0.00%
Unique unmatched postcodes: 0
[]


In [20]:
missing_df = (
    all_pp.isna().sum()
      .rename("n_missing")
      .to_frame()
      .assign(
         pct_missing = lambda df: (df["n_missing"] / len(all_pp) * 100).round(2)
      )
      .sort_values("pct_missing", ascending = False)
)
print(missing_df)

                   n_missing  pct_missing
txn_id                     0          0.0
postcode                   0          0.0
price                      0          0.0
property_type              0          0.0
old_new                    0          0.0
duration                   0          0.0
ppd_category_type          0          0.0
record_status              0          0.0
country_code               0          0.0
region_code                0          0.0
latitude                   0          0.0
longitude                  0          0.0
police_force_area          0          0.0
imd_score                  0          0.0
LSOA_code                  0          0.0


In [21]:
all_pp = all_pp.drop_duplicates(subset = "txn_id", keep = "first")

In [22]:
before = len(all_pp)
print(f"{before:,} rows before dropping duplicates")

4,024,048 rows before dropping duplicates


In [23]:
all_pp = all_pp.query("record_status == 'A'") # only keeps the rows that satisfy the boolean condition
after = len(all_pp)
print(f"Kept {after:,}", f"dropped {before - after:,} rows")

Kept 4,024,048 dropped 0 rows


In [24]:
required = [
    "postcode", 
    "country_code", 
    "region_code", 
    "latitude", 
    "longitude", 
    "police_force_area", 
    "imd_score", 
    "LSOA_code", 
]

In [25]:
all_pp_london = all_pp.query("region_code == 'E12000007'") # London

In [29]:
all_missing = all_pp_london[required].isna().all(axis = 1).sum()
print(f"{all_missing:,} rows are completely blank in all {len(required)} columns.")

0 rows are completely blank in all 8 columns.


In [30]:
any_missing = all_pp_london[required].isna().any(axis = 1).sum()
print(f"{any_missing:,} rows are missing at least one of those columns.")

0 rows are missing at least one of those columns.


In [31]:
pattern = (
    all_pp_london[required]
      .isna()
      .astype(int)
      .assign(_count = lambda df: df.sum(axis = 1))
      ._count
      .value_counts()
      .sort_index()
)
print("Number of rows by how many of the required columns are missing:")
print(pattern)

Number of rows by how many of the required columns are missing:
_count
0    451737
Name: count, dtype: int64


In [32]:
miss_count = all_pp_london[required].isna().sum(axis = 1)
clean_pp = all_pp_london.loc[miss_count <= 6].reset_index(drop = True)

In [ ]:
print(
    f"Dropped {(miss_count >= 7).sum():,} rows missing 7–8 fields, "
    f"kept {len(clean_pp):,} rows."
)

In [33]:
missing_df = (
    clean_pp.isna().sum()
      .rename("n_missing")
      .to_frame()
      .assign(
         pct_missing = lambda df: (df["n_missing"] / len(clean_pp) * 100).round(2)
      )
      .loc[required]
)
print(missing_df)

                   n_missing  pct_missing
postcode                   0          0.0
country_code               0          0.0
region_code                0          0.0
latitude                   0          0.0
longitude                  0          0.0
police_force_area          0          0.0
imd_score                  0          0.0
LSOA_code                  0          0.0


In [ ]:
from scipy.spatial import cKDTree

# 1) Build a KD‐tree of all ONSPD postcodes that have LSOA, region, police, etc.
geo = onspd[["postcode", "latitude", "longitude", "LSOA_code", "region_code", "police_force_area"]].dropna()
coords = geo[["latitude", "longitude"]].values # index on latitude & longitude
tree = cKDTree(coords)

In [ ]:
# 2) Mask the 3 rows in clean_pp that still need imputation:
mask = clean_pp[required].isna().any(axis = 1)

In [ ]:
# 3) Query the nearest known point for each of those rows
query_pts = clean_pp.loc[mask, ["latitude", "longitude"]].values
_, idxs = tree.query(query_pts, k = 1)

In [ ]:
# 4) Grab the metadata from geo at those indices
nearest = geo.iloc[idxs].reset_index(drop = True)

In [ ]:
for col in ["LSOA_code", "region_code", "police_force_area"]:
    clean_pp.loc[mask, col] = nearest[col].values

In [ ]:
print(clean_pp[required].isna().sum())

In [35]:
import os
processed = project_root / "data" / "clean-pp"

In [36]:
processed.mkdir(parents = True, exist_ok = True)

In [37]:
csv_fp = processed / "clean_pp.csv"
parquet_fp = processed / "clean_pp.parquet"

In [38]:
clean_pp.to_csv(csv_fp, index = False)

In [39]:
clean_pp.to_parquet(parquet_fp, index = False)